In [1]:
def validate_integer(value):
    try:
        value = int(value)
    except ValueError:
        raise TypeError('Value must be an integer.')
    if isinstance(value, int) and value <= 0:
        raise ValueError('Value must be greater than zero.')
    return value


In [2]:
validate_integer('hello')

TypeError: Value must be an integer.

In [5]:
validate_integer(-1)

ValueError: Value must be greater than zero.

In [38]:
def validate_string(value, name):
    value = str(value)
    if not len(value):
        raise ValueError(f'{name} must not be empty.')
    return value

def validate_integer(value, zero_allowed=False):
    try:
        value = int(value)
    except ValueError:
        raise TypeError('Value must be an integer.')
    if zero_allowed:
        if value < 0:
            raise ValueError('Value must not be negative.') 
    else:
        if value <= 0:
            raise ValueError('Value must be greater than zero.') 
    return value

In [146]:
# inventory.py

class Resource:

    def __init__(self, name, manufacturer, total=0, allocated=0):
        self._name = validate_string(name, 'Name')
        self._manufacturer = validate_string(manufacturer, 'Manufacturer')
        self._total = 0
        self._allocated = 0
        self.purchase(total, zero_allowed=True)
        self.claim(allocated, zero_allowed=True)

    @property
    def name(self):
        return self._name

    @property
    def manufacturer(self):
        return self._manufacturer

    @property
    def total(self):
        return self._total

    def purchase(self, qty, zero_allowed=False):
        self._total += validate_integer(qty, zero_allowed=zero_allowed)
        self._remaining = None
        return f'New total quantity: {self.total}'

    @property
    def allocated(self):
        return self._allocated

    def claim(self, qty, zero_allowed=False):
        qty = validate_integer(qty, zero_allowed=zero_allowed)
        if qty > self.remaining:
            raise ValueError('Not enough available stock.')
        self._allocated += qty
        self._remaining = None
        return f'New allocated quantity: {self.allocated}'

    @property
    def remaining(self):
        if self._remaining is None:
            self._remaining = self.total - self.allocated
        return self._remaining

    def freeup(self, qty):
        qty = validate_integer(qty)
        self._allocated -= qty
        self._remaining = None
        return f'New allocated quantity: {self.allocated}'

    def died(self, qty):
        qty = validate_integer(qty)
        if qty > self.total:
            raise ValueError('Cannot kill more than total stock quantity.')
        else:
            self._total -= qty
            self._remaining = None
            if self.total <= self.allocated:
                self._allocated = self.total
        print(f'{qty} unit' + ('s' if qty > 1 else '') + ' died.'
               f'\nNew total: {self.total}'
               f'\nAllocated: {self.allocated}'
               f'\nRemaining: {self.remaining}')

    def __str__(self):
        return self.name

    def __repr__(self):
        return f'{self.__class__.__name__} (name={self.name}, manufacturer={self.manufacturer})'


    


In [147]:
cpu = Resource('CPU', 'AMD', 100, 20)

In [148]:
str(cpu)

'CPU'

In [149]:
cpu

Resource (name=CPU, manufacturer=AMD)

In [150]:
t = 1

In [151]:
print('unit' + ('s' if t > 1 else ''))

unit


In [160]:
cpu.died(2)

2 units died.
New total: 2
Allocated: 2
Remaining: 0


In [75]:
print(f'test{t}'+'\ntest')

test1
test
